In [212]:
import pandas as pd

# Load the DataFrame from the pickle file
df = pd.read_pickle('Q1_XER9.pkl')

In [213]:
import pandas as pd

# Assuming you have a DataFrame named df
# Modify the display option to show full content of all columns
pd.set_option('display.max_colwidth', None)

# Access and print the row
print(df.iloc[-1])

# Reset the display option to its default value
pd.reset_option('display.max_colwidth')

_id                                                                                                                                                                                                                                                                                                                                                                                                                     65b0e03b38f7d3b322401845
measurements    {'4': {'pv0': 0.694735169, 'pv1': 0.788486063, 'pv2': 0.381972194, 'pv3': 2.497282505}, '22': {'pv0': 0.499801666, 'pv1': 0.67623651, 'pv2': 0.20180957, 'pv3': 2.495594978}, '24': {'pv0': 0.507855177, 'pv1': 0.683502078, 'pv2': 0.199316531, 'pv3': 2.49612999}, '29': {'pv0': 0.569297016, 'pv1': 0.624747455, 'pv2': 0.200586081, 'pv3': 2.497182369}, '31': {'pv0': -0.0625, 'pv1': -0.5, 'pv2': -1.125, 'pv3': -0.0625}}
meta                                                                                                                                  

In [214]:
for i in range(1, 11):
    print(df.iloc[-i]["measurements"].keys(), df.iloc[-i]["meta"]["uuid"])

dict_keys(['4', '22', '24', '29', '31']) 207137655863200
dict_keys(['2', '3', '15', '28']) 44332625541024
dict_keys(['5', '7', '9', '16', '18', '21']) 97186493085600
dict_keys(['1', '8', '12', '13', '14', '23', '26', '30']) 39714511164468
dict_keys(['6', '10', '11', '17', '19', '20', '25', '27']) 185284862261152
dict_keys(['4', '22', '24', '29', '31']) 207137655863200
dict_keys(['2', '3', '15', '28']) 44332625541024
dict_keys(['5', '7', '9', '16', '18', '21']) 97186493085600
dict_keys(['1', '8', '12', '13', '14', '23', '26', '30']) 39714511164468
dict_keys(['6', '10', '11', '17', '19', '20', '25', '27']) 185284862261152


In [215]:
unique_uuids = set()

for i in range(len(df)):
    unique_uuids.add(df.iloc[i]["meta"]["uuid"])

# Print all unique uuid values
for uuid in unique_uuids:
    print(uuid)

97186493085600
185284862261152
207137655863200
44332625541024
39714511164468


In [216]:
# Filter the original DataFrame for each span
df_span1 = df[df['meta'].apply(lambda x: x['uuid']) == 185284862261152].copy()
df_span2 = df[df['meta'].apply(lambda x: x['uuid']) == 44332625541024].copy()
df_span3 = df[df['meta'].apply(lambda x: x['uuid']) == 39714511164468].copy()
df_span4 = df[df['meta'].apply(lambda x: x['uuid']) == 207137655863200].copy()
df_span5 = df[df['meta'].apply(lambda x: x['uuid']) == 97186493085600].copy()

df_spans = [df_span1, df_span2, df_span3, df_span4, df_span5]

In [288]:
def flatten_dict(d, parent_key='', sep='_'):
    """
    Recursively flattens a nested dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f'{parent_key}{sep}{k}' if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def flatten_row(row):
    # Flatten 'measurements'
    measurements = row['measurements']
    flattened_measurements = {}
    for sensor, values in measurements.items():
        for pv_key, pv_value in values.items():
            flattened_measurements[f'{sensor}.{pv_key}'] = pv_value

    # Flatten 'meta' with nested dictionaries
    meta = row['meta']
    flattened_meta = flatten_dict(meta, parent_key='meta')

    # Flatten 'time' with nested dictionaries
    time = row['time']
    flattened_time = flatten_dict(time, parent_key='time')

    # Combine all flattened data
    flattened_data = {**flattened_measurements,
                      **flattened_meta, **flattened_time}
    return pd.Series(flattened_data)


# Apply the updated flattening function to each DataFrame in df_spans
df_spans_flattened = []
for df in df_spans:
    df_flattened = df.apply(flatten_row, axis=1)
    df_final = pd.concat(
        [df.drop(['measurements', 'meta', 'time'], axis=1), df_flattened], axis=1)
    df_spans_flattened.append(df_final)

In [218]:
for df in df_spans_flattened:
    cols_to_drop = []
    for col in df.columns:
        if col[0].isdigit():  # Check if column starts with a number
            if col.startswith('31'):
                # For columns starting with 31, drop if it ends with pv0 or pv3
                if col.endswith('pv0') or col.endswith('pv3'):
                    cols_to_drop.append(col)
            elif col.startswith('3') or col.startswith('4'):
                # For columns starting with 3 or 4, drop only if it ends with pv3
                if col.endswith('pv3'):
                    cols_to_drop.append(col)
            else:
                # For other columns, drop if it ends with pv2 or pv3
                if col.endswith('pv2') or col.endswith('pv3'):
                    cols_to_drop.append(col)
    df.drop(columns=cols_to_drop, inplace=True)


columns_to_drop = ['meta_modem_localIP', 'meta_modem_modemInfo',
                   'meta_modem_operator', 'meta_modem_simCCID', "meta_location"]

for df in df_spans_flattened:
    df.drop(columns=columns_to_drop, inplace=True)

In [219]:
for i in df_spans_flattened:
    print(i.columns)

Index(['_id', '10.pv0', '10.pv1', '11.pv0', '11.pv1', '17.pv0', '17.pv1',
       '19.pv0', '19.pv1', '20.pv0', '20.pv1', '25.pv0', '25.pv1', '27.pv0',
       '27.pv1', '6.pv0', '6.pv1', 'meta_bootCount', 'meta_devices',
       'meta_errors', 'meta_modem_signalQuality', 'meta_power_battery_V',
       'meta_power_solar_V', 'meta_uuid', 'meta_version', 'time_server_UTC',
       'time_server_epoch'],
      dtype='object')
Index(['_id', '15.pv0', '15.pv1', '2.pv0', '2.pv1', '28.pv0', '28.pv1',
       '3.pv0', '3.pv1', '3.pv2', 'meta_bootCount', 'meta_devices',
       'meta_errors', 'meta_modem_signalQuality', 'meta_power_battery_V',
       'meta_power_solar_V', 'meta_uuid', 'meta_version', 'time_server_UTC',
       'time_server_epoch'],
      dtype='object')
Index(['_id', '1.pv0', '1.pv1', '12.pv0', '12.pv1', '13.pv0', '13.pv1',
       '14.pv0', '14.pv1', '23.pv0', '23.pv1', '26.pv0', '26.pv1', '30.pv0',
       '30.pv1', '30.pv2', '8.pv0', '8.pv1', 'meta_bootCount', 'meta_devices',
       '

In [220]:
df_spans_flattened[-1].iloc[0]

_id                         6540fb0d38f7d3b3223edb86
16.pv0                                      0.712053
16.pv1                                      0.622148
18.pv0                                      0.647477
18.pv1                                      0.731319
21.pv0                                      0.476454
21.pv1                                      0.481008
5.pv0                                        0.58086
5.pv1                                       0.383469
7.pv0                                        0.59398
7.pv1                                       0.404381
9.pv0                                       0.724579
9.pv1                                       0.623893
meta_bootCount                                   743
meta_devices                                       6
meta_errors                                        0
meta_modem_signalQuality                          21
meta_power_battery_V                           3.994
meta_power_solar_V                            

In [221]:
import plotly.graph_objs as go
import plotly.subplots as sp

df_plot = df_spans_flattened[0]

df_plot['time_server_UTC'] = pd.to_datetime(df_plot['time_server_UTC'])

# Create subplots
fig_combined = sp.make_subplots(rows=1, cols=1)

average_set = "6H"
# Assuming df_spans_flattened is a list of DataFrames
for index, df_plot in enumerate(df_spans_flattened):
    df_plot['time_server_UTC'] = pd.to_datetime(df_plot['time_server_UTC'])

    # Resample to get hourly averages
    hourly_avg = df_plot.resample(average_set, on='time_server_UTC')[
        'meta_power_battery_V'].mean()

    # Add a trace for each DataFrame with a unique legend name
    trace = go.Scatter(x=hourly_avg.index, y=hourly_avg.values,
                       mode='lines', name=f'Pole mosta - {index+1}')
    fig_combined.add_trace(trace)

# Update layout to place legend on top, change background color, use "T10" color scheme, and set aspect ratio to 3:2
fig_combined.update_layout(title='Hladiny baterií systému',
                           xaxis_title='Dátum', yaxis_title=f'Priemerná hodnota batérie - {average_set} interval',
                           xaxis=dict(tickangle=45),
                           legend=dict(orientation='h', yanchor='bottom',
                                       y=1.02, xanchor='right', x=1),
                           plot_bgcolor='#EEEEEE',
                           colorway=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                                     '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'],
                           width=900, height=500)  # Set width and height for 3:2 aspect ratio

# Show the combined plot
fig_combined.show()

### Span 1

In [289]:
import pandas as pd
import plotly.express as px

df_sens = df_spans_flattened[0]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '10.pv1'].mean() * 25 - 12.065081

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x0A', mode='lines', line_shape='spline', line_color='#16CC62')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 1, snímač 1',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


df_sens = df_spans_flattened[0]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '6.pv1'].mean() * 25 - 13.712718

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x06', mode='lines', line_shape='spline', line_color='#196EE6')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 1, snímač 2',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


df_sens = df_spans_flattened[0]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '19.pv0'].mean() * 25 - 14.705731

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x13', mode='lines', line_shape='spline', line_color='#E6B219')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 1, snímač 3',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


df_sens = df_spans_flattened[0]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '17.pv0'].mean() * 25 - 14.801795

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x11', mode='lines', line_shape='spline', line_color='#E6196E')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 1, snímač 4',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


df_sens = df_spans_flattened[0]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '11.pv1'].mean() * 25 - 14.280117

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x0B', mode='lines', line_shape='spline', line_color='#19C3E6')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 1, snímač 5',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


df_sens = df_spans_flattened[0]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '20.pv1'].mean() * 25 - 15.970398

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '20.pv0'].mean() * 25 - 10.464455

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x14', mode='lines', line_shape='spline', line_color='#116149')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 1, snímač 6',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


df_sens = df_spans_flattened[0]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '27.pv0'].mean() * 25 - 9.490378

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '27.pv1'].mean() * 25 - 9.961429

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x1B', mode='lines', line_shape='spline', line_color='#E65D19')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 1, snímač 7',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


df_sens = df_spans_flattened[0]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '25.pv0'].mean() * 25 - 9.659468

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '25.pv1'].mean() * 25 - 14.169122

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x19', mode='lines', line_shape='spline', line_color='#8A27A5')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 1, snímač 8',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()

### Span 2

In [253]:
import pandas as pd
import plotly.express as px

df_sens = df_spans_flattened[1]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '15.pv0'].mean() * 25 - 18.07682

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '15.pv1'].mean() * 25 - 18.760167

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x0F', mode='lines', line_shape='spline', line_color='#16CC62')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 2, snímač 1',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


#############################################################################
#############################################################################

df_sens = df_spans_flattened[1]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '3.pv0'].mean() * 25 - 13.419626

# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '3.pv1'].mean() * 25 - 11.759914

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x03', mode='lines', line_shape='spline', line_color='#196EE6')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 2, snímač 2',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


#############################################################################
#############################################################################

df_sens = df_spans_flattened[1]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '2.pv0'].mean() * 25 - 9.667387

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '2.pv1'].mean() * 25 - 12.092839

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x02', mode='lines', line_shape='spline', line_color='#E6B219')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 2, snímač 3',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()

#############################################################################
#############################################################################

df_sens = df_spans_flattened[1]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '28.pv0'].mean() * 25 - 15.315167

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '28.pv1'].mean() * 25 - 14.672931

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x1C', mode='lines', line_shape='spline', line_color='#E6196E')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 2, snímač 4',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()

### Span 3

In [256]:
import pandas as pd
import plotly.express as px

span = 2
df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '12.pv0'].mean() * 25 - 15.496291

# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '12.pv1'].mean() * 25 - 16.182252

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x0C', mode='lines', line_shape='spline', line_color='#16CC62')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 3, snímač 1',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '30.pv0'].mean() * 25 - 10.861233

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '30.pv1'].mean() * 25 - 8.933031

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x1E', mode='lines', line_shape='spline', line_color='#196EE6')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 3, snímač 2',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '1.pv0'].mean() * 25 - 11.889166

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '1.pv1'].mean() * 25 - 14.688036

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x01', mode='lines', line_shape='spline', line_color='#E6B219')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 3, snímač 3',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()

#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '14.pv0'].mean() * 25 - 12.138769

# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '14.pv1'].mean() * 25 - 12.817644

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x0E', mode='lines', line_shape='spline', line_color='#E6196E')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 3, snímač 4',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()

#############################################################################
#############################################################################


df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '13.pv0'].mean() * 25 - 20.030608

# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '13.pv1'].mean() * 25 - 18.425565

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x0D', mode='lines', line_shape='spline', line_color='#19C3E6')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 3, snímač 5',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()
#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '26.pv0'].mean() * 25 - 13.115771

# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '26.pv1'].mean() * 25 - 15.748022

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x1A', mode='lines', line_shape='spline', line_color='#116149')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 3, snímač 6',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()

#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '23.pv0'].mean() * 25 - 11.197258

# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '23.pv1'].mean() * 25 - 9.686405

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x17', mode='lines', line_shape='spline', line_color='#E65D19')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 3, snímač 7',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()

#############################################################################
#############################################################################


df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '8.pv0'].mean() * 25 - 3.245663

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '8.pv1'].mean() * 25 - 13.464751

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x08', mode='lines', line_shape='spline', line_color='#8A27A5')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 3, snímač 8',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()
#############################################################################
#############################################################################

### Span 4


In [257]:
import pandas as pd
import plotly.express as px

span = 3
df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '29.pv0'].mean() * 25 - 14.231817

# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '29.pv1'].mean() * 25 - 15.583086

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x1D', mode='lines', line_shape='spline', line_color='#16CC62')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 4, snímač 1',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '24.pv0'].mean() * 25 - 12.77958

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '24.pv1'].mean() * 25 - 17.086151

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x18', mode='lines', line_shape='spline', line_color='#196EE6')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 4, snímač 2',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '22.pv0'].mean() * 25 - 12.510103

# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '22.pv1'].mean() * 25 - 16.883504

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x16', mode='lines', line_shape='spline', line_color='#E6B219')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 4, snímač 3',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()

#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '4.pv0'].mean() * 25 - 17.291336

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '4.pv1'].mean() * 25 - 19.664619

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x04', mode='lines', line_shape='spline', line_color='#E6196E')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 4, snímač 4',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()

#############################################################################
#############################################################################

### Span 5

In [260]:
import pandas as pd
import plotly.express as px

span = 4
df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '21.pv0'].mean() * 25 - 11.910764

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '21.pv1'].mean() * 25 - 12.023627

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x15', mode='lines', line_shape='spline', line_color='#16CC62')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 4, snímač 1',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '18.pv0'].mean() * 25 - 16.189261

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '18.pv1'].mean() * 25 - 18.281011

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x12', mode='lines', line_shape='spline', line_color='#196EE6')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 4, snímač 2',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '7.pv0'].mean() * 25 - 14.850544

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '7.pv1'].mean() * 25 - 10.108321

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x07', mode='lines', line_shape='spline', line_color='#E6B219')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 4, snímač 3',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()

#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '16.pv0'].mean() * 25 - 17.795829

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '16.pv1'].mean() * 25 - 15.551453

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x10', mode='lines', line_shape='spline', line_color='#E6196E')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 4, snímač 4',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()

#############################################################################
#############################################################################


df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '5.pv0'].mean() * 25 - 14.521514

# df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
#     '5.pv1'].mean() * 25 - 9.584026

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x05', mode='lines', line_shape='spline', line_color='#19C3E6')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 4, snímač 5',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()
#############################################################################
#############################################################################

df_sens = df_spans_flattened[span]
df_ref = df_spans_flattened[3]

average_set = "6H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])
df_ref['time_server_UTC'] = pd.to_datetime(df_ref['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '9.pv0'].mean() * 25 - 18.116027

df_sens_resampled = df_sens.resample(average_set, on='time_server_UTC')[
    '9.pv1'].mean() * 25 - 15.59752

df_ref_resampled = df_ref.resample(average_set, on='time_server_UTC')[
    '4.pv2'].mean() * 25 - 9.560792

# Calculate the difference between the two Series
difference_resampled = df_sens_resampled - df_ref_resampled

# Sort difference_resampled by values
sorted_difference_resampled = difference_resampled.sort_values()

# Calculate min and max differences
min_difference = sorted_difference_resampled.iloc[0]
max_difference = sorted_difference_resampled.iloc[-1]

# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_sens_resampled.index, y=df_sens_resampled, name='0x09', mode='lines', line_shape='spline', line_color='#116149')
fig_combined_pv.add_scatter(
    x=df_ref_resampled.index, y=df_ref_resampled, name='referencia', mode='lines', line_shape='spline', line_color='#333333')

# Add text annotations for min and max differences
fig_combined_pv.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_combined_pv.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.3f} mm', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

fig_combined_pv.update_layout(title='Pole 4, snímač 6',
                              xaxis_title='Dátum',
                              yaxis_title='Lineárny posun [mm]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))

# Show the plot
fig_combined_pv.show()


### Temperatures

In [271]:
import pandas as pd
import plotly.express as px

df_sens = df_spans_flattened[3]

average_set = "12H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_temp_1 = df_sens.resample(average_set, on='time_server_UTC')[
    '31.pv1'].mean()

df_temp_2 = df_sens.resample(average_set, on='time_server_UTC')[
    '31.pv2'].mean()


# Create a Plotly line plot for '0x0A' and 'reference' with custom line colors
fig_combined_pv = px.line()
fig_combined_pv.add_scatter(
    x=df_temp_1.index, y=df_temp_1, name='Spodná pásnica', mode='lines', line_shape='spline', line_color='#FF6600')
fig_combined_pv.add_scatter(
    x=df_temp_2.index, y=df_temp_2, name='Horná pásnica', mode='lines', line_shape='spline', line_color='#0066FF')


fig_combined_pv.update_layout(title='Teplota konštrukcie',
                              xaxis_title='Dátum',
                              yaxis_title='Teplota [°C]',
                              legend=dict(orientation='h', yanchor='bottom',
                                          y=1.02, xanchor='right', x=1),
                              plot_bgcolor='#EEEEEE',
                              width=900, height=500,
                              margin=dict(l=50, r=25, t=50, b=50))


# Show the plot
fig_combined_pv.show()

In [286]:
import pandas as pd
import plotly.express as px

df_sens = df_spans_flattened[3]

average_set = "1H"

# Convert 'time_server_UTC' to datetime and resample
df_sens['time_server_UTC'] = pd.to_datetime(df_sens['time_server_UTC'])

# Resample and compute mean for the relevant columns
df_temp_1 = df_sens.resample(average_set, on='time_server_UTC')[
    '31.pv1'].mean()

df_temp_2 = df_sens.resample(average_set, on='time_server_UTC')[
    '31.pv2'].mean()

# Calculate the difference between temperatures
temperature_difference = df_temp_2 - df_temp_1

# Sort difference_resampled by values
sorted_difference = temperature_difference.sort_values()

# Calculate min and max differences
min_difference = sorted_difference.iloc[0]
max_difference = sorted_difference.iloc[-1]

# Create a Plotly line plot for temperature differences
fig_temperature_difference = px.line()
fig_temperature_difference.add_scatter(
    x=temperature_difference.index, y=temperature_difference, name='Teplotný rozdiel', mode='lines', line_shape='spline', line_color='#333333')

fig_temperature_difference.update_layout(title='Teplotný spád',
                                         xaxis_title='Dátum',
                                         yaxis_title='Teplotný rozdiel [°C]',
                                         legend=dict(orientation='h', yanchor='bottom',
                                                     y=1.02, xanchor='right', x=1),
                                         plot_bgcolor='#EEEEEE',
                                         width=900, height=500,
                                         margin=dict(l=50, r=25, t=50, b=50))

# Add text annotations for min and max differences
fig_temperature_difference.add_annotation(
    text=f'Minimálny rozdiel: {min_difference:.1f} °C', xref='paper', yref='paper', x=0.95, y=0.95, showarrow=False)
fig_temperature_difference.add_annotation(
    text=f'Maximálny rozdiel: {max_difference:.1f} °C', xref='paper', yref='paper', x=0.95, y=0.90, showarrow=False)

# Show the plot
fig_temperature_difference.show()